# Infer CYTOSOL - part 3

--------------

## OBJECTIVE:  Infer sub-cellular component #3: CYTOSOL  in order to understand interactome 

To measure shape, position, and size of the cytosol of the cell body -- the soma minus the nucleus.    

Dependencies:
The CYTOSOL inference rely on the NUCLEI AND SOMA inference.  Therefore all of the sub-cellular objects rely on this segmentation.





# IMPORTS

In [ ]:

# this needs to be organzied to explain the imports
from pathlib import Path
import os
from collections import defaultdict

import numpy as np
import scipy
from scipy import ndimage as ndi

import napari

# function for core algorithm
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, hole_filling
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters, img_as_float
from skimage import morphology

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label
# # package for io 
# from aicsimageio import AICSImage

from napari.utils.notebook_display import nbscreenshot

#import .infer_subc.base
from infer_subc.base import *


: 

# IMAGE PROCESSING Objective 3:  infer CYTOSOL
  [OUTLINE: Objective #2](#summary-of-objectives)
## summary of steps

INPUT
- labeled NUCLEI (objective #1)
- labeled SOMA (objective #2)

PRE-PROCESSING
- erode Nuclei (shrink)
- CORE-PROCESSIN
  - XOR(NUCLEI, SOMA)

- POST-PROCESSING
  - N/A

OUTPUT
- mask of CYTOSOL
- CYTOSOL object (intensity)



# INPUT

In [ ]:
###################
# INPUT
###################

# load these if we don't have them
# NU_object
# SO_object

NU_object_file, meta_dict_t = read_input_image("out_data/NU_object.ome.tiff")

SO_object_file, meta_dict_t = read_input_image("out_data/SO_object.ome.tiff")
SO_labels_file, meta_dict_t = read_input_image("out_data/SO_labels.ome.tiff")


In [ ]:
###################
# PRE_PROCESSING
###################

#NU_eroded1 = morphology.binary_erosion(NU_object,  footprint=morphology.ball(3) )
NU_eroded = morphology.binary_erosion(NU_object)  
###################
# CORE_PROCESSING
###################

# logical  SO - NU_eroded
CY_object = np.logical_and(SO_NU_mask,~NU_eroded)
CY_object2 = np.logical_and(SO_mask,~NU_eroded)

# use SOMA flourescence signal
CY_signal = SO_signal.copy()
CY_signal[~CY_object] = 0

# use total flourescence signal
CY_signal_all = total_flourescence_scaled.copy()
CY_signal_all[~CY_object] = 0

In [ ]:

CY_object_filen = export_ome_tiff(CY_object, meta_in, "CY_object", "./out_data/", curr_chan=0)
#CY_signal_filen = export_ome_tiff(CY_signal, meta_in, "CY_signal", "./out_data/", curr_chan=0)


viewer.add_image(
    CY_object,
    scale=scale,
    opacity=0.3,
)    

viewer.add_image(
    CY_object2,
    scale=scale,
    opacity=0.3,
)    
